In [18]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image as image_utils
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input

In [19]:
input_tensor = Input(shape=(220,293, 3))
#include top是要不要fully connection
model=keras.applications.xception.Xception(include_top=False, weights='imagenet',
                                    input_tensor=input_tensor,
                                    pooling=None, classes=15)

In [20]:
x = model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(output_dim=128, activation='relu')(x)
x=Dropout(p=0.2)(x)
x = Dense(output_dim=128, activation='relu')(x)


# and a logistic layer -- let's say we have 200 classes


predictions = Dense(output_dim=15,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)

/home/jason/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  after removing the cwd from sys.path.
/home/jason/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.2)`
  """
/home/jason/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  
/home/jason/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=15)`
  if sys.path[0] == '':


In [21]:
len(model.layers)

137

In [22]:
#如果想鎖住本來model裡面的權重
'''for layer in model.layers:
    layer.trainable = False''' 
for layer in model.layers[:120]:
   layer.trainable = False
for layer in model.layers[120:139]:
   layer.trainable = True
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [23]:
import os
from keras.callbacks import EarlyStopping, ModelCheckpoint
if not os.path.isdir(os.getcwd()):
    os.makedirs(os.getcwd())
model_path = os.path.join(os.getcwd(),'cat_dog_autosave_classification.h5')
checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)

#validation_loss訓練超過五次都沒進步，主要是避免overfitting
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

In [24]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   rotation_range=30,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   vertical_flip=False)
#width_shift_range=0.1,height_shift_range=0.1,

In [25]:
training_set = train_datagen.flow_from_directory('/home/jason/Desktop/where_am_i/train',
                                                 target_size = (220, 293),
                                                 batch_size = 32,#可以改
                                                 class_mode = 'categorical')


Found 2985 images belonging to 15 classes.


In [26]:
from os import listdir
from os.path import isfile, isdir, join

# 指定要列出所有檔案的目錄
mypath = "/home/jason/Desktop/where_am_i"

# 取得所有檔案與子目錄名稱
files = listdir(mypath)

# 以迴圈處理
for f in files:
  # 產生檔案的絕對路徑
  fullpath = join(mypath, f)
  # 判斷 fullpath 是檔案還是目錄
  if isfile(fullpath):
    print("檔案：", f)
  elif isdir(fullpath):
    print("目錄：", f)

檔案： test.ipynb
檔案： tensorflow.ipynb
檔案： keras2_answer.csv
檔案： whereami-dl-tensorflow.ipynb
檔案： 期中.ipynb
檔案： testset_img_list.csv
檔案： train_img_list.csv
目錄： .ipynb_checkpoints
目錄： aia-picture-classification1
檔案： .DS_Store
檔案： train_img_list_label.csv
檔案： mid_term_mapping.txt
檔案： the_answer.csv
檔案： img-submission.csv
檔案： keras.ipynb
目錄： testset
目錄： train


In [27]:
model.fit_generator(training_set,
                         samples_per_epoch = 2985,
                         nb_epoch = 30)

/home/jason/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jason/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=93, epochs=30)`
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/30
93/93 [==============================] - 30s 320ms/step - loss: 0.9973 - acc: 0.6856
Epoch 2/30
93/93 [==============================] - 28s 306ms/step - loss: 0.4809 - acc: 0.8422
Epoch 3/30
93/93 [==============================] - 28s 305ms/step - loss: 0.3992 - acc: 0.8686
Epoch 4/30
93/93 [==============================] - 29s 309ms/step - loss: 0.3406 - acc: 0.8953
Epoch 5/30
93/93 [==============================] - 29s 310ms/step - loss: 0.3358 - acc: 0.8953
Epoch 6/30
93/93 [==============================] - 29s 310ms/step - loss: 0.3265 - acc: 0.8968
Epoch 7/30
93/93 [==============================] - 29s 311ms/step - loss: 0.2501 - acc: 0.9240
Epoch 8/30
93/93 [==============================] - 29s 311ms/step - loss: 0.2399 - acc: 0.9235
Epoch 9/30
93/93 [==============================] - 29s 307ms/step - loss: 0.2159 - acc: 0.9326
Epoch 10/30
93/93 [==============================] - 29s 307ms/step - loss: 0.2129 - acc: 0.9363
Epoch 11/30
93/93 [====================

In [11]:
#預測
street 13
kitchen 8
mountain 10
insidecity 7
livingroom 9
opencountry 11
CALsuburb 0
PARoffice 1
highway 5
coast 3
bedroom 2
tallbuilding 14
industrial 6
forest 4
store 12

#真實
CALsuburb, 9
PARoffice, 7
bedroom, 12
coast, 10
forest, 4
highway, 14
industrial, 2
insidecity, 3
kitchen, 0
livingroom, 5
mountain, 8
opencountry, 6
store, 11
street, 1
tallbuilding, 13

SyntaxError: invalid syntax (<ipython-input-11-4d16553960c8>, line 2)

In [28]:
import glob
import numpy as np

All_picture=[]
path =r'/home/jason/Desktop/where_am_i/testset' # use your path
allFiles = glob.glob(path + "/*.jpg")
test_=[]
for picture in allFiles:
    test_image_1 = image_utils.load_img(picture, target_size=(220, 293))
    test_image_1 = image_utils.img_to_array(test_image_1)
    test_image_1 = np.expand_dims(test_image_1, axis=0)
    test_image_1 =test_image_1 /255
    new=model.predict(test_image_1)
    new=np.argmax(new)
    test_.append(new)
answer=[]
for x in test_:
    if x==13:
        x=1
    elif x==8:
        x=0
    elif x==10:
        x=8
    elif x==7:
        x=3
    elif x==9:
        x=5
    elif x==11:
        x=6
    elif x==0:
        x=9
    elif x==1:
        x=7
    elif x==5:
        x=14
    elif x==3:
        x=10
    elif x==2:
        x=12
    elif x==14:
        x=13
    elif x==6:
        x=2
    elif x==4:
        x=4
    elif x==12:
        x=11
    answer.append(x)
len(answer)

1500

In [29]:
import pandas as pd
result_csv=pd.read_csv('img-submission.csv')

In [30]:
result_csv.head()

,id,class
0,c117693e1cf24a5232090d1548cb11d4e5ea0df65680c4...,1
1,96baacc2e97886a998807ce197574821a6dc83c227c746...,1
2,f9bd26db23eb9b544ca78be79b11b4d1259e802885861d...,1
3,b42bcd8e6645fcc2ac40ee44b7dc8d74a77081d0aea7a1...,1
4,5bc53cef9168882f0ff67a81b3e7269f62b7fd5343d06d...,1


In [32]:
first_columns=[]
for x in allFiles:
    hi=x.split('/')[6].split('.')[0]
    first_columns.append(hi)
result_csv['id']=first_columns 
result_csv['class']=answer

In [33]:
result_csv.head()

,id,class
0,0b6ee55106786db0eacf7cab333b3dbd20f9f0e85a0859...,6
1,3663fc4e1a0119ab3aa8d53e12ce2007f2a35aa0b1f9f3...,6
2,e7a0975f76e6e9fe820a07f47080d264b960e79605890b...,8
3,7bae1d6cec7feb538eb7ff985191cacf96e122eaf4be21...,14
4,4a46ae60618eb7bb43d7c227d2c606927549e5e09c0e32...,12


In [ ]:
'''import os 
os.chdir('/home/jovyan')
result_csv=pd.read_csv('img-submission.csv')
result_csv['class']=answer'''

In [34]:
result_csv.to_csv('keras3_answer.csv',index=False)

In [ ]:
from keras.preprocessing import image as image_utils
test_answer=[]
result_csv=pd.read_csv('img-submission.csv')
for data_real in result_csv['id']:
    test_answer.append('//data/examples/may_the_4_be_with_u/where_am_i/testset/'+data_real+'.jpg')
test_=[]
for picture in test_answer:
    test_image_1 = image_utils.load_img(picture, target_size=(224, 224))
    test_image_1 = image_utils.img_to_array(test_image_1)
    test_image_1 = np.expand_dims(test_image_1, axis=0)
    test_image_1 = test_image_1/255
    new=model.predict(test_image_1)
    new=np.argmax(new)
    test_.append(new)

In [ ]:
from keras.preprocessing import image as image_utils
test_answer=[]
result_csv=pd.read_csv('img-submission.csv')
for data_real in result_csv['id']:
    test_answer.append('//data/examples/may_the_4_be_with_u/where_am_i/testset/'+data_real+'.jpg')
test_=[]
for picture in test_answer:
    test_image_1 = image_utils.load_img(picture, target_size=(224, 224))
    test_image_1 = image_utils.img_to_array(test_image_1)
    test_image_1 = np.expand_dims(test_image_1, axis=0)
    test_image_1 = test_image_1/255
    new=model.predict(test_image_1)
    new=np.argmax(new)
    test_.append(new)

In [254]:
image = Image.open('image_0001.jpg')
type(image)
test_image_1 = image_utils.load_img(picture)
test_image_1 = image_utils.img_to_array(test_image_1)

In [256]:
test_image_1.shape

(220, 330, 3)

In [ ]:
13 1
8 0
10 8
7 3
9 5
11 6
0 9
1 7
5 14
3 10
2 12
14 13
6 2
4 4
12 11